In [ ]:
from skimage import io,filters,data
import numpy as np
img=data.coins()

In [ ]:
m,n=img.shape
for i in range (0,m):
    for j in range (0,n):
        if img[i,j]>127:
            img[i,j]=255
        else:
            img[i,j]=0
io.imshow(img)

In [ ]:
def erosion(image):
    
    m,n=image.shape
    img1=np.zeros([m,n])
    for x in range (1,m-1):
        for y in range (1,n-1):
            sum=image[x-1,y-1]/255+image[x-1,y]/255+image[x-1,y+1]/255+image[x,y-1]/255+image[x,y]/255+image[x,y+1]/255+image[x+1,y-1]/255+image[x+1,y]/255+image[x+1,y+1]/255
            if sum==9:
                img1[x,y]=255
            else:
                img1[x,y]=0
    img1=img1.astype(np.uint8)            
    return img1

img2=erosion(img)
io.imshow(img2)

In [ ]:
#Dialation
def dilation(image):
    
    m,n=image.shape
    img1=np.zeros([m,n])
    for x in range (1,m-1):
        for y in range (1,n-1):
            sum=image[x-1,y-1]/255+image[x-1,y]/255+image[x-1,y+1]/255+image[x,y-1]/255+image[x,y]/255+image[x,y+1]/255+image[x+1,y-1]/255+image[x+1,y]/255+image[x+1,y+1]/255
            sum=sum
            if sum>0:
                img1[x,y]=255
            else:
                img1[x,y]=0
    img1=img1.astype(np.uint8)            
    return img1
img3=dilation(img)
io.imshow(img3)

In [ ]:
#opening
cur=erosion(img)
opening=dilation(cur)
io.imshow(opening)

In [ ]:
#closing
cur1=dilation(img)
closing=erosion(cur1)
io.imshow(closing)

In [ ]:
#frequency domain codes
from skimage import data,io,filters
from skimage import io
from skimage import color
from skimage.restoration import denoise_nl_means, estimate_sigma

import numpy as np
from numpy.fft import fft, fftfreq, ifft

from scipy import ndimage as nd
from scipy.fft import fft, ifft
from scipy import fftpack
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from math import sqrt
from math import exp
from skimage.util import random_noise

In [ ]:
image=data.camera()

In [ ]:
#printing FFT, Centered FFT, Decentralized FFT,
plt.figure(figsize=(8, 6), constrained_layout=False)

img_fft = np.fft.fft2(image)
img_fftshift = np.fft.fftshift(img_fft)
img_ifftshit = np.fft.ifftshift(img_fftshift)
img_ifft = np.fft.ifft2(img_ifftshit)


plt.subplot(231), plt.imshow(image, "gray"), plt.title("Original Image")
plt.subplot(232), plt.imshow(np.log(1+np.abs(img_fft)), "gray"), plt.title("Spectrum, FFT")
plt.subplot(233), plt.imshow(np.log(1+np.abs(img_fftshift)), "gray"), plt.title("Centered Spectrum")
plt.subplot(234), plt.imshow(np.log(1+np.abs(img_ifftshit)), "gray"), plt.title("Decentralized IFFT")
plt.subplot(235), plt.imshow(np.abs(img_ifft), "gray"), plt.title("Reversed Image")

plt.show()


In [ ]:
#Low pass
def low_pass(img):
    m,n=img.shape
    D0=50
    H=np.zeros((m,n),dtype=np.float32)
    for x in range(m):
        for y in range(n):
            d=sqrt((x-m/2)*(x-m/2)+(y-n/2)*(y-n/2))
            if(d<=D0):
                H[x,y]=1
            else:
                H[x,y]=0
    return H;



#High Pass
def high_pass(img):
    m,n=img.shape
    D0=50
    H=np.zeros((m,n),dtype=np.float32)
    for x in range(m):
        for y in range(n):
            d=sqrt((x-m/2)*(x-m/2)+(y-n/2)*(y-n/2))
            if(d<=D0):
                H[x,y]=0
            else:
                H[x,y]=1
    return H;


#Butter worth
def BUTTER(img):
    m,n=img.shape
    D0=30
    D1=60
    H=np.zeros((m,n),dtype=np.float32)
    for x in range(m):
        for y in range(n):
            d=sqrt((x-m/2)*(x-m/2)+(y-n/2)*(y-n/2))
            if(d>=D0 and d<=D1):
                H[x,y]=0
            else:
                H[x,y]=1
    return H;    

In [ ]:
img_fftshift=low_pass(img_fftshift)
io.imshow(img_fftshift)

In [ ]:
#frequency domain filters High, low and Butter worth
def Frequency_domain(image,filt):
    
    plt.figure(figsize=(8, 6), constrained_layout=False)

    img_fft = np.fft.fft2(image)
    img_fftshift = np.fft.fftshift(img_fft)
    if(filt=="L"):
        img_fftshift=img_fftshift*low_pass(img_fftshift)
    elif(filt=="H"):
        img_fftshift=img_fftshift*high_pass(img_fftshift)
    else:
        img_fftshift=img_fftshift*BUTTER(img_fftshift)
        
    img_ifftshit = np.fft.ifftshift(img_fftshift)
    img_ifft = np.fft.ifft2(img_ifftshit)


    plt.subplot(231), plt.imshow(image, "gray"), plt.title("Original Image")
    plt.subplot(232), plt.imshow(np.log(1+np.abs(img_fft)), "gray"), plt.title("Spectrum, FFT")
    plt.subplot(233), plt.imshow(np.log(1+np.abs(img_fftshift)), "gray"), plt.title("Centered Spectrum")
    plt.subplot(234), plt.imshow(np.log(1+np.abs(img_ifftshit)), "gray"), plt.title("Decentralized IFFT")
    plt.subplot(235), plt.imshow(np.abs(img_ifft), "gray"), plt.title("Reversed Image")

    plt.show()
    


In [ ]:
#adding noise
noise_img = random_noise(image, mode='s&p',amount=0.3)
io.imshow(noise_img)

In [ ]:
frequency_fitered_image=Frequency_domain(noise_img,"L")
frequency_fitered_image=Frequency_domain(noise_img,"H")
frequency_fitered_image=Frequency_domain(noise_img,"B")